In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_completables_progress* function**:
* Input: players_info dictionary 
* Displays bar chart showing for each player the progress achieved in the different completables of the game as well as in the total game

In [ ]:
def vis_completables_progress():
    out=interactive(getProgressPlayerPerCompletable,
                      xaxis=xaxis,
                      order=order,
                    xaxisPerCompletable=True)
    md_text=widgets.HTML("Bar chart with progress of players in each completable")
    return widgets.VBox([md_text,out])

def getProgressPlayerPerCompletable(xaxis, xaxisPerCompletable, order):
    #get alls players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completableSelectedProgress=getSublistSelectedFromUnionListSelected(completable_progress_list, checkboxesCompletableSelected.get_selected_options())
    completab={}
    for player in playersSelected:
        if not player in completab.keys():
            completab[player]={}
        if players_info[player]["completables_progress"] != {}:
            for c in completableSelectedProgress:
                if c in players_info[player]["completables_progress"].keys():
                    completab[player][c]=players_info[player]["completables_progress"][c]
                else:
                    completab[player][c]=0
    # progress of each player per completable
    players_progress = {}
    for player in completab.keys():
        for c in completableSelectedProgress:
            if not c in players_progress.keys():
                players_progress[c] = {}
            if c in completab[player].keys():
                players_progress[c][player] = completab[player][c]*100
            else:
                players_progress[c][player] = 0
    ordered_progress = orderDictByDictValues(players_progress,order)
    pos, bar_width = getPosDictMultiBarChart(ordered_progress,playersSelected)
    if not xaxisPerCompletable:
        players_progress = reorganiseDictBySubKeys(players_progress)
    nbgraphshown, players_progress,N = organizeDictPerxAxisSelected(completableSelectedProgress,xaxis,players_progress)
    if xaxisPerCompletable:
        pos = organizeDictPosPerDataxaxis(xaxis,pos)
    else:
        pos = organizeDictPosPerDataLabels(xaxis,pos)
    for n in range(nbgraphshown):
        n_groups = len(players_progress[n])
        if xaxisPerCompletable:
            players_progress[n] = reorganiseDictBySubKeys(players_progress[n])
        if n_groups>0:
            constructMultiBarWithVerticalLines(players_progress,n,
                                               completableSelectedProgress,
                                               pos, bar_width, N,
                                               xaxisPerCompletable)
        else:
            noDataToFillVis(100, True)
        plt.xlabel("Completable")
        plt.ylabel("Progress (in %)")
        plt.title("Progress of players per completable (in %)")
        plt.show()